# Antarctic Circumnavigation Expedition Cruise Track data processing

## GLONASS and Trimble GPS data

Follow the steps as described here: http://epic.awi.de/48174/

Import relevant packages

In [ ]:
import pandas as pd
import csv
import MySQLdb
import datetime
import math
import numpy as np

### STEP 1 - Extract data from database

Import data from a database table into a dataframe

In [ ]:
def get_data_from_database(query, db_connection):
    
    dataframe = pd.read_sql(query, con=db_connection)

    return dataframe

**GPS data**

In [205]:
query_trimble = 'select * from ship_data_gpggagpsfix where device_id=63 order by date_time;'
password = input()

ace


In [206]:
db_connection = MySQLdb.connect(host = 'localhost', user = 'ace', passwd = password, db = 'ace2016', port = 3306); 

gpsdb_df = get_data_from_database(query_trimble, db_connection)
#gpsdb_df_opt = optimise_dataframe(gpsdb_df_opt)

Preview data

In [207]:
gpsdb_df.sample(5)

,id,date_time,latitude,longitude,fix_quality,number_satellites,horiz_dilution_of_position,altitude,altitude_units,geoid_height,geoid_height_units,device_id,measureland_qualifier_flags_id
5882620,11543483,2017-03-04 03:26:27.060,-54.017306,-37.436617,5,7,0.128,6.222,M,10.293,M,63,1
5322454,10442819,2017-02-25 15:49:49.790,-53.170333,-70.906562,5,9,0.128,-2.612,M,12.840,M,63,1
1125075,1777333,2017-01-04 02:47:40.480,-50.307577,67.568075,5,8,0.128,-4.750,M,38.827,M,63,1
951043,1430416,2017-01-02 02:26:28.890,-47.551142,55.587443,5,8,0.128,3.304,M,39.930,M,63,1
6310384,12397105,2017-03-09 02:16:57.710,-59.147568,-18.050111,5,6,0.128,-3.909,M,21.486,M,63,1


Number of data points output

In [208]:
len(gpsdb_df)

8534827

Dates and times covered by the Trimble data set: 

In [209]:
print("Start date:", gpsdb_df['date_time'].min())
print("End date:", gpsdb_df['date_time'].max())

Start date: 2016-12-21 06:59:14.440000
End date: 2017-04-11 08:07:13.600000


Output the data into monthly files (to be able to plot them to visually screen the obvious outliers).

In [210]:
gpsdb_df.dtypes

id                                         int64
date_time                         datetime64[ns]
latitude                                 float64
longitude                                float64
fix_quality                                int64
number_satellites                          int64
horiz_dilution_of_position               float64
altitude                                 float64
altitude_units                            object
geoid_height                             float64
geoid_height_units                        object
device_id                                  int64
measureland_qualifier_flags_id             int64
dtype: object

In [211]:
gpsdb_df['date_time_day'] = gpsdb_df['date_time'].dt.strftime('%Y-%m-%d')

In [212]:
gpsdb_df.sample(5)

,id,date_time,latitude,longitude,fix_quality,number_satellites,horiz_dilution_of_position,altitude,altitude_units,geoid_height,geoid_height_units,device_id,measureland_qualifier_flags_id,date_time_day
5246426,10290587,2017-02-24 18:39:42.660,-53.170322,-70.906566,5,8,0.128,-4.072,M,12.840,M,63,1,2017-02-24
5474098,10746094,2017-02-27 09:57:16.030,-55.017352,-59.600214,5,6,0.133,-0.995,M,16.495,M,63,1,2017-02-27
3316191,6455759,2017-02-02 10:15:35.840,-65.366140,157.762000,5,9,0.128,1.022,M,-45.270,M,63,1,2017-02-02
5490432,10778763,2017-02-27 14:29:30.270,-55.110360,-57.902141,5,7,0.128,1.566,M,16.066,M,63,1,2017-02-27
7906691,16641956,2017-04-04 00:22:44.170,22.895949,-18.696221,5,8,0.128,-0.559,M,27.143,M,63,1,2017-04-04


In [ ]:
days = gpsdb_df.groupby('date_time_day')
for day in days.groups:
    path = '/home/jen/ace_trimble_gps_' + str(day) + '.csv'
    days.get_group(day).to_csv(path, index=False)

**GLONASS data**

In [ ]:
query_glonass = 'select * from ship_data_gpggagpsfix where device_id=64;'

password = input()


In [ ]:
db_connection = MySQLdb.connect(host = 'localhost', user = 'ace', passwd = password, db = 'ace2016', port = 3306); 

glonassdb_df = get_data_from_database(query_glonass, db_connection)

Preview data

In [ ]:
glonassdb_df.sample(5)

Number of data points output

In [ ]:
len(glonassdb_df)

Dates and times covered by the GLONASS data set: 

In [ ]:
print("Start date:", glonassdb_df['date_time'].min())
print("End date:", glonassdb_df['date_time'].max())

Output the data into monthly files (to be able to plot them to visually screen the obvious outliers).

In [ ]:
glonassdb_df['date_time_day'] = glonassdb_df['date_time'].dt.strftime('%Y-%m-%d')

In [ ]:
glonassdb_df.sample(5)

In [ ]:
days = glonassdb_df.groupby('date_time_day')
for day in days.groups:
    path = '/home/jen/ace_glonass_' + str(day) + '.csv'
    days.get_group(day).to_csv(path, index=False)

### STEP 2 - Visual inspection

### Trimble GPS

Data in the form of the daily csv files, were imported into QGIS mapping software in order to manually visually inspect the data points. This was done at a resolution of 1:100,000.

There were no obvious outlying points. 
Number of points flagged as outliers: 0

The following sections were identified as unusual and have been classified in the table below: 
| Start date and time (UTC) | End date and time (UTC) | Potential problem |
|---------|-----------|----------|
| 2016-12-21 10:15:46.620 | 2016-12-21 10:16:35.620 | Overlapping track |
| 2016-12-21 07:37:16.470 | 2016-12-21 07:40:02.470 | Overlapping track |
| 2016-12-22 04:43:10.010 | 2016-12-22 04:43:14.010 | Overlapping track |
| 2016-12-22 09:38:40.270 | 2016-12-22 09:38:41.270 | Strange gap |
| 2016-12-23 00:15:22.060 | 2016-12-23 00:15:33.060 | Overlapping track |
| 2016-12-23 04:57:57.310 | 2016-12-23 04:57:58.310 | Large gap |
| 2016-12-23 05:05:05.310 | 2016-12-23 05:11:43.540 | Overlapping track |
| 2016-12-23 05:30:01.560 | 2016-12-23 05:30:04.560 | Large gap |
| 2016-12-25 06:04:50.900 | 2016-12-25 06:09:52.980 | Overlapping track |
| 2016-12-29 12:57:59.390 | 2016-12-29 14:57:34.730 | Strange diversion |
| 2016-12-30 04:53:24.840 | 2016-12-30 08:51:12.670 | Strange diversion, missing data |
| 2016-12-31 00:51:37.580 | 2016-12-31 00:51:39.580 | |
| 2017-01-01 11:54:29.820 | 2017-01-01 11:54:41.820 | Overlapping track |
| 2017-01-01 22:51:51.680 | 2017-01-01 22:54:41.700 | Strange deflection |
| 2017-01-01 22:56:05.700 | 2017-01-01 22:59:57.700 | Strange deflection |
| 2017-01-04 00:34:30.360 | 2017-01-04 00:34:33.360 | Large move |
| 2017-01-13 08:26:40.850 | 2017-01-13 08:30:36.840 | Strange deflection |
| 2017-03-11 06:47:22.300 | 2017-03-11 08:21:40.970 | Strange deflection |
| 2017-03-16 17:51:36.490 | 2017-03-16 18:12:21.470 | Gap with jump |
| 2017-03-17 15:29:07.620 | 2017-03-17 15:29:10.620 | Gap with jump |
| 2017-03-18 04:07:31.290 | 2017-03-18 04:07:32.290 | Gap with jump |
| 2017-03-18 12:43:22.100 | 2017-03-18 12:43:42.100 | Overlapping track |
| 2017-03-18 13:01:04.100 | 2017-03-18 19:09:28.440 | Large gap with large time difference |
| 2017-03-24 10:13:24.720 | 2017-03-24 10:13:25.720 | Gap with jump |
| 2017-03-25 10:07:08.990 | 2017-03-25 10:07:15.990 | Gap with jump |
| 2017-03-26 22:25:49.940 | 2017-03-26 22:25:50.940 | Gap with jump |
These points have not been flagged but will be returned to later on in the processing. 

### GLONASS

Data in the form of the daily csv files, were imported into QGIS mapping software in order to manually visually inspect the data points. This was done at a resolution of 1:100,000.

There were no obvious outlying points. 
Number of points flagged as outliers: 0
    
The following sections were identified as unusual and have been classified in the table below: 
| Start date and time (UTC) | End date and time (UTC) | Potential problem |
|---------|-----------|----------|
| 2017-04-06 15:26:30 | 2017-04-06 15:26:32 | Gap with jump |
| 2017-04-06 18:22:07 | 2017-04-06 18:22:09 | Gap with jump |
| 2017-04-06 21:41:21 | 2017-04-06 21:41:45 | Strange deflection |
| 2017-04-07 15:25:06 | 2017-04-07 15:25:13 | Strange deflection |
| 2017-04-08 04:12:44 | 2017-04-08 04:13:27 | Strange deflection |
| 2017-04-08 05:01:34 | 2017-04-08 05:01:49 | Strange deflection |
| 2017-04-08 18:42:12 | 2017-04-08 18:43:04 | Strange deflection |
| 2017-04-08 18:56:02 | 2017-04-08 18:57:27 | Strange deflection |
| 2017-04-08 19:00:26 | 2017-04-08 19:01:03 | Strange deflection |
| 2017-04-08 19:05:31 | 2017-04-08 19:05:46 | Strange deflection |
| 2017-04-10 02:43:40 | 2017-04-10 02:44:25 | Strange deflection |

### STEP 3 - Motion data correction

### STEP 4 - Automated data filtering

Each data point will be compared with the one before and after to automatically filter out points that are out of the conceivable range of the ship's movement.'

The second of two consecutive points to be flagged as "likely incorrect" when any of the following cases occur: 
    - speed between two points >= 20 knots
    - acceleration between two points >= 1 ms^-2
    - direction between two points >= 5 degrees

In [213]:
# Test
df_test = gpsdb_df.head(10000)
df_test.head(5)

,id,date_time,latitude,longitude,fix_quality,number_satellites,horiz_dilution_of_position,altitude,altitude_units,geoid_height,geoid_height_units,device_id,measureland_qualifier_flags_id,date_time_day
0,613,2016-12-21 06:59:14.440,-35.838077,18.030717,5,9,0.128,3996.895,M,31.545,M,63,1,2016-12-21
1,614,2016-12-21 06:59:15.440,-35.838145,18.030720,5,9,0.128,3997.434,M,31.545,M,63,1,2016-12-21
2,615,2016-12-21 06:59:16.440,-35.838212,18.030725,5,9,0.128,3997.843,M,31.545,M,63,1,2016-12-21
3,616,2016-12-21 06:59:17.440,-35.838281,18.030731,5,9,0.128,3997.911,M,31.545,M,63,1,2016-12-21
4,617,2016-12-21 06:59:18.440,-35.838350,18.030737,5,9,0.128,3997.550,M,31.545,M,63,1,2016-12-21


In [214]:
def get_location(datetime, position_df):
    """Create a tuple of the date_time, latitude and longitude of a location in a dataframe from a given date_time."""
    
    latitude = position_df[position_df.date_time == datetime].latitude.item()
    longitude = position_df[position_df.date_time == datetime].longitude.item()
    
    location = (datetime, latitude, longitude)
    
    return location

In [215]:
def calculate_distance(origin, destination):
    """Calculate the haversine or great-circle distance in metres between two points with latitudes and longitudes, where they are known as the origin and destination."""
    
    datetime1, lat1, lon1 = origin
    datetime2, lat2, lon2 = destination
    radius = 6371  # km
    
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) \
                                                  * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c # Distance in km
    d_m = d*1000 # Distance in metres
    
    return d_m

In [216]:
def knots_two_points(origin, destination):
    """Calculate the speed in knots between two locations which are dictionaries containing latitude, longitude and date_time."""
    
    distance = calculate_distance(origin, destination)
    
    datetime1_timestamp, lat1, lon1 = origin
    datetime2_timestamp, lat2, lon2 = destination
    #datetime1 = datetime.datetime.strptime(datetime_str1,"%Y-%m-%d %H:%M:%S.%f")
    #datetime2 = datetime.datetime.strptime(datetime_str2,"%Y-%m-%d %H:%M:%S.%f")
    
    datetime1 = datetime1_timestamp.timestamp()
    datetime2 = datetime2_timestamp.timestamp()
    
    seconds = abs((datetime1) - (datetime2))
    #seconds = abs((datetime_str1)-(datetime_str2)).total_seconds()
    conversion = 3600/1852 # convert 1 ms-1 to knots (nautical miles per hour; 1 nm = 1852 metres)
    speed_knots = (distance/seconds) * conversion
    
    if seconds > 0:
        return speed_knots
    else:
        return "N/A"

In [ ]:
destination = (Timestamp('2016-12-21 06:59:14.440000'), -35.8380768456667, 18.0307165761667)
origin = (Timestamp('2016-12-21 06:59:14.440000'), -35.8380768456667, 18.0307165761667)
knots_two_points(origin, destination)

In [217]:
def set_utc(date_time):
    """Set the timezone to be UTC."""
    utc = datetime.timezone(datetime.timedelta(0))
    date_time = date_time.replace(tzinfo=utc)
    return date_time

In [256]:
def analyse_speed(position_df):
    """Analyse the cruise track to ensure each point lies within a reasonable distance and direction from the previous point."""
    
    total_data_points = len(position_df)
    
    earliest_date_time = position_df['date_time'].min()
    latest_date_time = position_df['date_time'].max()

    current_date = earliest_date_time

    previous_position = get_location(earliest_date_time, position_df)
    datetime_previous, latitude_previous, longitude_previous = previous_position
      
    count_speed_errors = 0
    
    line_number = -1
    for position in position_df.itertuples():
        line_number += 1
        if line_number == 0:
            continue

        current_position = position[2:5]
        row_index = position[0]
        
        #print(current_position)
        speed_knots = knots_two_points(previous_position, current_position)

        error_message = ""

        if speed_knots == "N/A":
            error_message = "No speed?"
            position_df.at[row_index, 'measureland_qualifier_flag_speed'] = 9
            position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_speed'] = 9
            print(position_df['id' == row_index])
        elif speed_knots >= 20:
            error_message += "** Too fast **"
            #print(row_id)
            #print(position_df[position_df['id'] == row_id])
            position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_speed'] = 4
            count_speed_errors += 1
        elif speed_knots < 20:
            position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_speed'] = 1

        if error_message != "":
            print("Error {} {}   ({:.4f}, {:.4f})   speed: {} knots".format(error_message, current_position[0], current_position[1], current_position[2], speed_knots))

        previous_position = current_position
        
    return count_speed_errors

In [257]:
def calculate_bearing(origin, destination):
    """Calculate the direction turned between two points."""
    
    datetime1, lat1, lon1 = origin
    datetime2, lat2, lon2 = destination
    
    dlon = math.radians(lon2 - lon1)
    
    bearing = math.atan2(math.sin(dlon) * math.cos(math.radians(lat2)), 
                         math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) 
                         - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(dlon))
    
    bearing_degrees = math.degrees(bearing)
    
    return bearing_degrees
    

In [258]:
def calculate_bearing_difference(current_bearing, previous_bearing):
    """Calculate the difference between two bearings, based on bearings between 0 and 360."""
    
    difference = current_bearing - previous_bearing

    while difference < -180:
        difference += 360
    while difference > 180:
        difference -= 360
    
    return difference

In [261]:
def analyse_course(position_df):
    """Analyse the change in the course between two points regarding the bearing and acceleration - these features need information from previous points."""
    
    total_data_points = len(position_df)
    
    earliest_date_time = position_df['date_time'].min()
    current_date = earliest_date_time
    
    previous_position = get_location(earliest_date_time, position_df)
    datetime_previous, latitude_previous, longitude_previous = previous_position
     
    previous_bearing = 0
    previous_speed_knots = 0

    count_bearing_errors = 0
    count_acceleration_errors = 0      

    line_number = -1
    for position in position_df.itertuples():
        line_number += 1
        if line_number == 0:
            continue
            
        current_position = position[2:5]
        row_index = position[0]

        # Calculate bearing and change in bearing
        current_bearing = calculate_bearing(previous_position, current_position)
        difference_in_bearing = calculate_bearing_difference(current_bearing, previous_bearing)
        
        # Calculate acceleration between two points
        current_speed_knots = knots_two_points(previous_position, current_position)

        time_difference = (current_position[0] - previous_position[0]).total_seconds()
        speed_difference_metres_per_sec = (current_speed_knots - previous_speed_knots) * (1852/3600) # convert knots to ms-1 
        
        if time_difference >0:
            acceleration = speed_difference_metres_per_sec / time_difference
        else:
            acceleration = 0

        # Print errors where data do not meet requirements
        error_message_bearing = ""
        error_message_acceleration = ""

        if difference_in_bearing == "N/A":
            error_message_bearing = "No bearing?"
            position_df.at[row_index, 'measureland_qualifier_flag_course'] = 9
            #position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_course'] = 9
            print(row_index)
            #print(position_df['id' == row_id])
        elif difference_in_bearing >= 5:
            error_message_bearing = "** Turn too tight **"
            position_df.at[row_index, 'measureland_qualifier_flag_course'] = 4
            #position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_course'] = 4
            print(row_index)
            #print(position_df['id' == row_id])
            count_bearing_errors += 1

        if error_message_bearing != "":
            print("Error:  {} {} ({:.4f}, {:.4f}) bearing change: {} degrees".format(error_message_bearing, current_position[0], current_position[1], current_position[2], difference_in_bearing))
        
        if acceleration =="N/A":
            error_message_acceleration = "No acceleration"
            position_df.at[row_index, 'measureland_qualifier_flag_acceleration'] = 9
            #position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_acceleration'] = 9
            print(row_index)
            #print(position_df['id' == row_id])
        elif acceleration > 1:
            count_acceleration_errors += 1
            error_message_acceleration = "** Acceleration to quick **"
            position_df.at[row_index, 'measureland_qualifier_flag_acceleration'] = 4            
            #position_df.loc[position_df['id'] == row_index, 'measureland_qualifier_flag_acceleration'] = 4
            print(row_index)
            #print(position_df['id' == row_id])
            
        if error_message_acceleration != "":
            print("Error:  {} {} ({:.4f}, {:.4f}) acceleration: {} ms-2".format(error_message_acceleration, current_position[0], current_position[1], current_position[2], acceleration))

        previous_position = current_position
        previous_bearing = current_bearing
        previous_speed_knots = current_speed_knots
        
    return (count_bearing_errors, count_acceleration_errors)

In [260]:
analyse_speed(df_test)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Error ** Too fast ** 2016-12-21 06:59:15.440000   (999.0000, 18.0307)   speed: 38906216.23416525 knots
Error ** Too fast ** 2016-12-21 06:59:16.440000   (-35.8382, 18.0307)   speed: 9761523.731244445 knots
Error ** Too fast ** 2016-12-21 07:38:34.470000   (-35.9918, 18.0735)   speed: 1508.480212836083 knots


KeyboardInterrupt: 

In [262]:
analyse_course(df_test)

1
Error:  ** Turn too tight ** 2016-12-21 06:59:15.440000 (999.0000, 18.0307) bearing change: 89.9999885944378 degrees
1
Error:  ** Acceleration to quick ** 2016-12-21 06:59:15.440000 (999.0000, 18.0307) acceleration: 20015086.79602057 ms-2
3
Error:  ** Turn too tight ** 2016-12-21 06:59:17.440000 (-35.8383, 18.0307) bearing change: 175.92133451484625 degrees
2360
Error:  ** Acceleration to quick ** 2016-12-21 07:38:34.470000 (-35.9918, 18.0735) acceleration: 766.005631429995 ms-2
2361
Error:  ** Turn too tight ** 2016-12-21 07:38:35.470000 (-35.9918, 18.0735) bearing change: 153.5151020875811 degrees


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(3, 2)

In [253]:
df_test.head()
#df_test.loc[df_test['id'] == 613, 'latitude'] = 99
#df_test.at[1, 'latitude'] = 999

,id,date_time,latitude,longitude,fix_quality,number_satellites,horiz_dilution_of_position,altitude,altitude_units,geoid_height,geoid_height_units,device_id,measureland_qualifier_flags_id,date_time_day,measureland_qualifier_flag_speed,measureland_qualifier_flag_course
0,613,2016-12-21 06:59:14.440,99.000000,18.030717,5,9,0.128,3996.895,M,31.545,M,63,1,2016-12-21,NaN,NaN
1,614,2016-12-21 06:59:15.440,999.000000,18.030720,5,9,0.128,3997.434,M,31.545,M,63,1,2016-12-21,4.0,4.0
2,615,2016-12-21 06:59:16.440,-35.838212,18.030725,5,9,0.128,3997.843,M,31.545,M,63,1,2016-12-21,1.0,NaN
3,616,2016-12-21 06:59:17.440,-35.838281,18.030731,5,9,0.128,3997.911,M,31.545,M,63,1,2016-12-21,1.0,NaN
4,617,2016-12-21 06:59:18.440,-35.838350,18.030737,5,9,0.128,3997.550,M,31.545,M,63,1,2016-12-21,1.0,NaN
